<a href="https://colab.research.google.com/github/cemreefe/cmpe493-project/blob/main/tfidf-cosine_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install xmltodict

import os
import tarfile
import pandas as pd
import xmltodict
import json
import io   
import math

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def read_file(path):
  with open(path, 'r') as f:
    return f.read()

In [4]:
if not os.path.exists('drive/MyDrive/CMPE/CMPE493'):
  os.makedirs('drive/MyDrive/CMPE/CMPE493')

In [5]:
if not os.path.exists('drive/MyDrive/CMPE/CMPE493/topics-rnd5.xml'):
  !curl https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml --output drive/MyDrive/CMPE/CMPE493/topics-rnd5.xml

if not os.path.exists('drive/MyDrive/CMPE/CMPE493/qrels-covid_d5_j0.5-5.txt'):
  !curl https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt --output drive/MyDrive/CMPE/CMPE493/qrels-covid_d5_j0.5-5.txt

if not os.path.exists('drive/MyDrive/CMPE/CMPE493/cord-19_2020-07-16.tar.gz'):
  !curl https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2020-07-16.tar.gz --output drive/MyDrive/CMPE/CMPE493/cord-19_2020-07-16.tar.gz

In [6]:
if not os.path.exists('2020-07-16'):
  tar = tarfile.open('drive/MyDrive/CMPE/CMPE493/cord-19_2020-07-16.tar.gz', "r:gz")
  tar.extractall()
  tar.close()

In [7]:
df_metadata = pd.read_csv('2020-07-16/metadata.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
del df_metadata['sha'], df_metadata['source_x'], df_metadata['doi'], df_metadata['pmcid'], df_metadata['pubmed_id'], df_metadata['license'], df_metadata['publish_time'], df_metadata['authors'], df_metadata['journal'], df_metadata['mag_id'], df_metadata['who_covidence_id'], df_metadata['arxiv_id'], df_metadata['pdf_json_files'], df_metadata['pmc_json_files'], df_metadata['url'], df_metadata['s2_id']

In [9]:
df_metadata[df_metadata['cord_uid'] == 'ug7v899j']

,cord_uid,title,abstract
0,ug7v899j,Clinical features of culture-proven Mycoplasma...,OBJECTIVE: This retrospective chart review des...


In [10]:
df_metadata.drop_duplicates(subset='cord_uid', keep='first', inplace=True)

In [11]:
df_metadata

,cord_uid,title,abstract
0,ug7v899j,Clinical features of culture-proven Mycoplasma...,OBJECTIVE: This retrospective chart review des...
1,02tnwd4m,Nitric oxide: a pro-inflammatory mediator in l...,Inflammatory diseases of the respiratory tract...
2,ejv2xln0,Surfactant protein-D and pulmonary host defense,Surfactant protein-D (SP-D) participates in th...
3,2b73a28n,Role of endothelin-1 in lung disease,Endothelin-1 (ET-1) is a 21 amino acid peptide...
4,9785vg6d,Gene expression in epithelial cells in respons...,Respiratory syncytial virus (RSV) and pneumoni...
...,...,...,...
192504,z4ro6lmh,Rapid radiological improvement of COVID-19 pne...,NaN
192505,hi8k8wvb,SARS E protein in phospholipid bilayers: an an...,Abstract We report on an anomalous X-ray refle...
192506,ma3ndg41,Italian Society of Interventional Cardiology (...,COVID‐19 pandemic raised the issue to guarante...
192507,wh10285j,"Nimble, Together: A Training Program's Respons...",NaN


In [12]:
topic_relevances = 'topic iter document_id judgement\n' + read_file('drive/MyDrive/CMPE/CMPE493/qrels-covid_d5_j0.5-5.txt')

df_relevances = pd.read_csv(  io.StringIO(topic_relevances)  , sep=" ")
del df_relevances['iter']

df_relevances

,topic,document_id,judgement
0,1,005b2j4b,2
1,1,00fmeepz,1
2,1,010vptx3,2
3,1,0194oljo,1
4,1,021q9884,1
...,...,...,...
69313,50,zvop8bxh,2
69314,50,zwf26o63,1
69315,50,zwsvlnwe,0
69316,50,zxr01yln,1


In [19]:
df_relevances[df_relevances['document_id'] == 'ug7v899j']

,topic,document_id,judgement


In [13]:
topics_obj = xmltodict.parse(read_file('drive/MyDrive/CMPE/CMPE493/topics-rnd5.xml'))
topics     = json.loads(json.dumps(topics_obj))

topics_dict = {}
for topic in topics['topics']['topic']:
  # a topic has the following fields:
  #  * @number
  #  * narrative
  #  * query
  #  * question
  topics_dict[topic['@number']] = topic['query'] + ' ' + topic['question'] + ' ' + topic['narrative']

In [14]:
len(topics_dict)

50

# Data so far

* `topics_dict` 
      has `topic-id` for keys, and topic description for values
* `df_relevances` 
      has the following three columns:
      topic	document-id	judgement
* `df_metadata`
      holds information about the documents
      has the following three columns (others are deleted):
      cord_uid	title	abstract


In [15]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
#stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
import numpy as np

docs = np.array(df_metadata)
contents = {}

for doc in docs:
  contents[doc[0]] = f'{doc[1]} {doc[2]}'

In [17]:
import string
import re
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
sw = stopwords.words('english')

def preprocess(s):
  s = s.casefold()
  s = s.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
  s = re.sub("\d+", "", s) #Delete numbers
  s = ' '.join([porter_stemmer.stem(word) for word in s.split() if word not in sw and word != 'nan'])
  return s

In [100]:
if os.path.exists('drive/MyDrive/CMPE/CMPE493'):
  doc_vectors = pickle.load(open("drive/MyDrive/CMPE/CMPE493/doc_vectors.pickle", "rb"))
  doc_vectorizer = pickle.load(open("drive/MyDrive/CMPE/CMPE493/doc_vectorizer.pickle", "rb"))

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

doc_vectorizer = TfidfVectorizer(preprocessor=preprocess)
doc_vectors = doc_vectorizer.fit_transform(list(contents.values()))

print(len(doc_vectorizer.vocabulary_))


145387


In [101]:
topic_vectors = doc_vectorizer.transform(list(topics_dict.values()))

In [28]:
import pickle
pickle.dump(doc_vectors, open("drive/MyDrive/CMPE/CMPE493/doc_vectors.pickle", "wb"))
pickle.dump(doc_vectorizer, open("drive/MyDrive/CMPE/CMPE493/doc_vectorizer.pickle", "wb"))

In [29]:
doc_vectors.shape

(191175, 145387)

In [30]:
topic_vectors.shape

(50, 145387)

In [24]:
doc_vectors_dict = {
    docs[i][0] : doc_vectors[i]
    for i in range(0,len(docs))
}

In [25]:
topic_vectors_dict = {
    key : topic_vectors[i]
    for i, key in enumerate(topics_dict.keys())
}

In [26]:
from sklearn.metrics.pairwise import cosine_similarity
#Takes random doc_id and prints most relevant topic
ex_document = df_metadata.sample().cord_uid.to_string().split()[1]
similarity = cosine_similarity(doc_vectors_dict[ex_document],topic_vectors).flatten()

print(ex_document)
#print(similarity)
print(np.max(similarity))
print(np.argmax(similarity))
print(contents[ex_document])
print(topics_dict[str(np.argmax(similarity) + 1)])


evqqed55
0.17073968476486598
32
The American Society for Virology--29th Annual Meeting. The American Society for Virology 29th Annual Meeting, held in Bozeman, MT, USA, included topics covering new vaccine technologies, delivery methods and treatments in the field of virology. This conference report highlights selected presentations on human norovirus (HuNoV), SARS coronavirus and Rift Valley fever virus vaccine technologies; programmed cell death-1 (PD1) blockade and HyperAcute alpha-Gal platform technology methods; aerosol vaccination delivery; novel technologies to produce influenza virus-like particles (VLP) in mammalian cell lines; and investigational human rotavirus vaccines.
coronavirus vaccine candidates What vaccine candidates are being tested for Covid-19? Seeking studies that discuss possible, but specific, COVID-19 vaccines. Includes articles from those describing the mechanisms of action of specific proposed vaccines to actual clinical trials, but excluding articles that d

In [59]:
sims = cosine_similarity(topic_vectors, doc_vectors)
sims.shape

In [ ]:
results = np.ndarray((len(contents)*int(len(topics_dict)/2),1))
results = list(results)

#Even results
for j,topic in enumerate(topics_dict.keys()):
  if(j%2 != 0):
    print(j)
    for i,doc_id in enumerate(contents.keys()):
      element = f'{topic} 0 {doc_id} 0 {sims[j][i]} 0'
      results[int(j/2)*len(contents.keys())+i] = element

In [98]:
results = np.ndarray((len(contents)*len(topics_dict),1))
results = list(results)

#All results
for j,topic in enumerate(topics_dict.keys()):
  for i,doc_id in enumerate(contents.keys()):
    element = f'{topic} 0 {doc_id} 0 {sims[j][i]} 0'
    results[j*len(contents.keys())+i] = element

In [99]:
results[:10]

['1 0 ug7v899j 0 0.0 0',
 '1 0 02tnwd4m 0 0.0 0',
 '1 0 ejv2xln0 0 0.0047753866594862065 0',
 '1 0 2b73a28n 0 0.0 0',
 '1 0 9785vg6d 0 0.038565120559759226 0',
 '1 0 zjufx4fo 0 0.0034306857186680154 0',
 '1 0 5yhe786e 0 0.0060537943467020355 0',
 '1 0 8zchiykl 0 0.0 0',
 '1 0 8qnrcgnk 0 0.003151106046927332 0',
 '1 0 jg13scgo 0 0.0 0']

In [96]:
with open('results.txt', 'w') as f:
    f.write('\n'.join(results))

qid iter   docno      rank  sim   run_id 

In [ ]:
topics_dict['1']

"coronavirus origin what is the origin of COVID-19 seeking range of information about the SARS-CoV-2 virus's origin, including its evolution, animal source, and first transmission into humans"

In [ ]:
contents['ug7v899j']

'Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (3

In [ ]:
a = set(preprocess(contents['ug7v899j']).split())
b = set(preprocess(topics_dict['1']).split())
a.intersection(b)

clinic featur cultur proven mycoplasma pneumonia infect king abdulaziz univers hospit jeddah saudi arabia object retrospect chart review describ epidemiolog clinic featur patient cultur proven mycoplasma pneumonia infect king abdulaziz univers hospit jeddah saudi arabia method patient posit pneumonia cultur respiratori specimen januari decemb identifi microbiolog record chart patient review result patient identifi requir admiss infect commun acquir infect affect age group common infant pre school children occur year round common fall spring three quarter patient comorbid twenti four isol associ pneumonia upper respiratori tract infect bronchiol cough fever malais common symptom crepit wheez common sign patient pneumonia crepit bronchial breath immunocompromis patient like non immunocompromis patient present pneumonia versu p patient pneumonia unev recoveri recov follow complic die pneumonia infect die due underli comorbid patient die pneumonia pneumonia comorbid conclus result similar 

set()

In [ ]:
sw.index('and')

63

In [ ]:
a_vector = doc_vectorizer.transform([contents['ug7v899j']])
b_vector = doc_vectorizer.transform([topics_dict['1']])
cosine_similarity(a_vector, b_vector)

array([[0.]])